In [24]:
!pip install pyspark

In [25]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import time

# Path to the MySQL JDBC driver JAR file
jdbc_driver_path = "/Users/timl/PythonWork/Spark/Spark Week/Archive/jars/mysql-connector-java-8.4.0.jar"

# Initialize Spark session with the MySQL JDBC driver
spark = SparkSession.builder \
    .appName("MySQLConnectionExample") \
    .config("spark.jars", jdbc_driver_path) \
    .getOrCreate()


# Define MySQL database connection properties
jdbc_url = "jdbc:mysql://localhost:3306/eventsim?unixSocket=/tmp/mysql.sock"
connection_properties = {
    "user": "timlinkous",  # Replace with your MySQL username
    "password": "zipcode1",  # Replace with your MySQL password
    "driver": "com.mysql.cj.jdbc.Driver"
}

# Specify the table name
table_name = "events"

# Attempt to read the table from MySQL
try:
    df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=connection_properties)
    print("Connection successful! Here are the first few rows of the table:")
    df.show(5)  # Display the first 5 rows
except Exception as e:
    print("Connection failed:")
    print(e)


24/08/24 14:27:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Connection successful! Here are the first few rows of the table:
+------+-----------------+-------------------+-------+---------+---------------+-----------+---------------+----------------------+-------------+------------+-------+--------------------+------------------+--------------------+------------------+--------------------+------------------+--------------------+-----------------+--------+------------------+-----------------+----+----------------+
|    id|       event_type|          timestamp|user_id|user_name|       user_age|user_gender|user_main_genre|user_subscription_plan|user_platform|  user_state|user_os|   user_current_page|          track_id|               title|           song_id|             release|         artist_id|         artist_mbid|      artist_name|duration|artist_familiarity|artist_hotttnesss|year|track_7digitalid|
+------+-----------------+-------------------+-------+---------+---------------+-----------+---------------+----------------------+-------------+--

In [26]:
pip install dash jupyter-dash plotly

Note: you may need to restart the kernel to use updated packages.


In [27]:
df.head()

Row(id=277133, event_type='streamed song', timestamp=datetime.datetime(2023, 8, 19, 12, 24), user_id=41, user_name='User41', user_age='45-54 years old', user_gender='Female', user_main_genre='country', user_subscription_plan='UnPaid', user_platform='Apple Music', user_state='Wisconsin', user_os='iOS', user_current_page='Artist Page', track_id='TRJAHAC128EF34C3C8', title="Pot_ Jet Packs_ and Peace in the Middle East (and drunk people yelling s''t on my CD) (Shiny Happy Jihad)", song_id='SOOGQHL12A6BD56BC0', release='Shiny Happy Jihad', artist_id='ARKQN8X1187B99221C', artist_mbid='7aeed70b-8768-4e19-9c5a-93dcba2f58e1', artist_name='Joe Rogan', duration=222.929, artist_familiarity=0.510993, artist_hotttnesss=0.379223, year=2007, track_7digitalid=761273)

In [28]:
df.columns

['id',
 'event_type',
 'timestamp',
 'user_id',
 'user_name',
 'user_age',
 'user_gender',
 'user_main_genre',
 'user_subscription_plan',
 'user_platform',
 'user_state',
 'user_os',
 'user_current_page',
 'track_id',
 'title',
 'song_id',
 'release',
 'artist_id',
 'artist_mbid',
 'artist_name',
 'duration',
 'artist_familiarity',
 'artist_hotttnesss',
 'year',
 'track_7digitalid']

In [29]:
pip install dash-bootstrap-components

Note: you may need to restart the kernel to use updated packages.


In [30]:
from pyspark.sql import functions as F
from pyspark.sql import Window
# Assuming your PySpark DataFrame is named 'df'
# Extract the hour from the timestamp if available, or generate a synthetic one
df = df.withColumn("hour_of_day", (F.rand() * 24).cast("integer"))  # Replace with actual timestamp column if available
# Aggregate the data by hour_of_day and user_main_genre
agg_df = df.groupBy("hour_of_day", "user_main_genre").agg(
    F.count("*").alias("play_count")
)
# Convert the aggregated PySpark DataFrame to Pandas
pandas_df = agg_df.toPandas()

# Main Dashboard

In [38]:
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
import plotly.express as px
from pyspark.sql import functions as F
import time

# Define charcoal gray color and white color
charcoal_gray = "#36454F"
white = "#FFFFFF"

# 1. Extract the hour from the timestamp
df = df.withColumn("hour_of_day", F.hour(F.col("timestamp")))

# 2. Aggregate the data by hour_of_day and user_main_genre
agg_df = df.groupBy("hour_of_day", "user_main_genre").agg(
    F.count("*").alias("play_count")
)

# 3. Convert the aggregated PySpark DataFrame to Pandas
pandas_df = agg_df.toPandas()

# 4. Sort the DataFrame by 'play_count' in ascending order
pandas_df = pandas_df.sort_values(by='play_count', ascending=False)

# 5. Calculate KPIs

# Top Song by Play Count
top_song = df.groupBy("title").agg(F.count("*").alias("play_count")).orderBy(F.desc("play_count")).first()

# Most Listened to Artist
top_artist = df.groupBy("artist_name").agg(F.count("*").alias("play_count")).orderBy(F.desc("play_count")).first()

# Total Play Count
total_play_count = df.count()

# Most Popular Genre
top_genre = df.groupBy("user_main_genre").agg(F.count("*").alias("play_count")).orderBy(F.desc("play_count")).first()

# Most Popular Subscription Plan
top_subscription_plan = df.groupBy("user_subscription_plan").agg(F.count("*").alias("user_count")).orderBy(F.desc("user_count")).first()

# Average Song Duration
average_song_duration = df.agg(F.avg("duration")).first()

# Most Active User
most_active_user = df.groupBy("user_name").agg(F.count("*").alias("play_count")).orderBy(F.desc("play_count")).first()

# 6. Create a bar chart using Plotly Express
fig = px.bar(pandas_df, x='hour_of_day', y='play_count', color='user_main_genre',
              title='Song Plays by Hour and Genre')

# Play Count by Age Group
age_agg_df = df.groupBy("user_age").agg(F.count("*").alias("play_count"))
age_pandas_df = age_agg_df.toPandas()
age_fig = px.bar(age_pandas_df, x='user_age', y='play_count', title='Play Count by Age Group')

# Play Count by Gender
gender_agg_df = df.groupBy("user_gender").agg(F.count("*").alias("play_count"))
gender_pandas_df = gender_agg_df.toPandas()
gender_fig = px.pie(gender_pandas_df, names='user_gender', values='play_count', title='Play Count by Gender')

# Play Count by Subscription Plan
subscription_agg_df = df.groupBy("user_subscription_plan").agg(F.count("*").alias("play_count"))
subscription_pandas_df = subscription_agg_df.toPandas()
subscription_fig = px.bar(subscription_pandas_df, x='user_subscription_plan', y='play_count', title='Play Count by Subscription Plan')

# Play Count by Platform
platform_agg_df = df.groupBy("user_platform").agg(F.count("*").alias("play_count"))
platform_pandas_df = platform_agg_df.toPandas()
platform_fig = px.bar(platform_pandas_df, x='user_platform', y='play_count', title='Play Count by Platform')

# Play Count by OS
os_agg_df = df.groupBy("user_os").agg(F.count("*").alias("play_count"))
os_pandas_df = os_agg_df.toPandas()
os_fig = px.bar(os_pandas_df, x='user_os', y='play_count', title='Play Count by OS')

# 7. Create the Dash app and use Bootstrap
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Music Listening Habits Dashboard"), className="mb-4")
    ]),

    # Display KPIs
dbc.Row([
    dbc.Col(dbc.Card([
        dbc.CardHeader(
            html.Strong("Top Song"),
            style={"backgroundColor": charcoal_gray, "color": white}
        ),
        dbc.CardBody(f"{top_song['title']} (Plays: {top_song['play_count']})")
    ])),
    dbc.Col(dbc.Card([
        dbc.CardHeader(
            html.Strong("Most Listened to Artist"),
            style={"backgroundColor": charcoal_gray, "color": white}
        ),    
        dbc.CardBody(f"{top_artist['artist_name']} (Plays: {top_artist['play_count']})")
    ])),
    dbc.Col(dbc.Card([
        dbc.CardHeader(
            html.Strong("Total Play Count"),
            style={"backgroundColor": charcoal_gray, "color": white}
        ),
        dbc.CardBody(f"{total_play_count}")
    ])),
    dbc.Col(dbc.Card([
        dbc.CardHeader(
            html.Strong("Most Popular Genre"),
            style={"backgroundColor": charcoal_gray, "color": white}
        ),
        dbc.CardBody(f"{top_genre['user_main_genre']} (Plays: {top_genre['play_count']})")
    ])),
    dbc.Col(dbc.Card([
        dbc.CardHeader(
            html.Strong("Most Popular Subscription Plan"),
            style={"backgroundColor": charcoal_gray, "color": white}
        ),
        dbc.CardBody(f"{top_subscription_plan['user_subscription_plan']} (Users: {top_subscription_plan['user_count']})")
    ])),
    dbc.Col(dbc.Card([
        dbc.CardHeader(
            html.Strong("Average Song Duration"),
            style={"backgroundColor": charcoal_gray, "color": white}
        ),
        dbc.CardBody(f"{round(average_song_duration['avg(duration)'] / 60, 2)} minutes")
    ])),
    dbc.Col(dbc.Card([
        dbc.CardHeader(
            html.Strong("Most Active User"),
            style={"backgroundColor": charcoal_gray, "color": white}
        ),
        dbc.CardBody(f"{most_active_user['user_name']} (Plays: {most_active_user['play_count']})")
    ])),
], className="mb-4"),

    # Existing Chart
    dbc.Row([
        dbc.Col(dcc.Graph(id='example-graph', figure=fig))
    ]),

    # New Charts
    dbc.Row([
        dbc.Col(dcc.Graph(id='age-play-count', figure=age_fig)),
        dbc.Col(dcc.Graph(id='gender-play-count', figure=gender_fig))
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='subscription-play-count', figure=subscription_fig)),
        dbc.Col(dcc.Graph(id='platform-play-count', figure=platform_fig))
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='os-play-count', figure=os_fig))
    ]),
], fluid=True)

# 8. Run the Dash app
if __name__ == '__main__':
    app.run_server(mode='external')


# Unused Dashboard

# Timer Updated Dashboard

In [32]:
from dash import Dash, dcc, html, Output, Input
import dash_bootstrap_components as dbc
import plotly.express as px
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Initialize SparkSession with JDBC driver
spark = SparkSession.builder \
    .appName("MusicDashboard") \
    .config("spark.jars", "/Users/timl/PythonWork/Spark/Spark Week/Archive/jars/mysql-connector-java-8.4.0.jar") \
    .getOrCreate()

# Define charcoal gray color and white color
charcoal_gray = "#36454F"
white = "#FFFFFF"


# Function to fetch and process data from MySQL
def fetch_data():
    global df
    df = spark.read.format("jdbc") \
        .option("url", "jdbc:mysql://localhost:3306/eventsim?unixSocket=/tmp/mysql.sock") \
        .option("dbtable", "events") \
        .option("user", "timlinkous") \
        .option("password", "zipcode1") \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .load()
    df = df.withColumn("hour_of_day", F.hour(F.col("timestamp")))

# Initial data fetch
fetch_data()

# 8. Create the Dash app and use Bootstrap
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Music Listening Habits Dashboard"), className="mb-4")
    ]),

    # Display KPIs
    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardHeader(
                html.Strong("Top Song"),
                style={"backgroundColor": charcoal_gray, "color": white}
            ),
            dbc.CardBody(id="top-song")
        ])),
        dbc.Col(dbc.Card([
            dbc.CardHeader(
                html.Strong("Most Listened to Artist"),
                style={"backgroundColor": charcoal_gray, "color": white}
            ),
            dbc.CardBody(id="top-artist")
        ])),
        dbc.Col(dbc.Card([
            dbc.CardHeader(
                html.Strong("Total Play Count"),
                style={"backgroundColor": charcoal_gray, "color": white}
            ),
            dbc.CardBody(id="total-play-count")
        ])),
        dbc.Col(dbc.Card([
            dbc.CardHeader(
                html.Strong("Most Popular Genre"),
                style={"backgroundColor": charcoal_gray, "color": white}
            ),
            dbc.CardBody(id="top-genre")
        ])),
        dbc.Col(dbc.Card([
            dbc.CardHeader(
                html.Strong("Most Popular Subscription Plan"),
                style={"backgroundColor": charcoal_gray, "color": white}
            ),
            dbc.CardBody(id="top-subscription-plan")
        ])),
        dbc.Col(dbc.Card([
            dbc.CardHeader(
                html.Strong("Average Song Duration"),
                style={"backgroundColor": charcoal_gray, "color": white}
            ),
            dbc.CardBody(id="avg-song-duration")
        ])),
        dbc.Col(dbc.Card([
            dbc.CardHeader(
                html.Strong("Most Active User"),
                style={"backgroundColor": charcoal_gray, "color": white}
            ),
            dbc.CardBody(id="most-active-user")
        ])),
    ], className="mb-4"),

    # Existing Chart
    dbc.Row([
        dbc.Col(dcc.Graph(id='example-graph'))
    ]),

    # New Charts
    dbc.Row([
        dbc.Col(dcc.Graph(id='age-play-count')),
        dbc.Col(dcc.Graph(id='gender-play-count'))
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='subscription-play-count')),
        dbc.Col(dcc.Graph(id='platform-play-count'))
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='os-play-count'))
    ]),

    # Interval component for periodic updates
    dcc.Interval(
        id='interval-component',
        interval=10*1000,  # in milliseconds (every 10 seconds)
        n_intervals=0,
        max_intervals=6  # Run only 6 times
    )
], fluid=True)

# Callbacks to update the dashboard
@app.callback(
    [Output('top-song', 'children'),
     Output('top-artist', 'children'),
     Output('total-play-count', 'children'),
     Output('top-genre', 'children'),
     Output('top-subscription-plan', 'children'),
     Output('avg-song-duration', 'children'),
     Output('most-active-user', 'children'),
     Output('example-graph', 'figure'),
     Output('age-play-count', 'figure'),
     Output('gender-play-count', 'figure'),
     Output('subscription-play-count', 'figure'),
     Output('platform-play-count', 'figure'),
     Output('os-play-count', 'figure')],
    [Input('interval-component', 'n_intervals')]
)
def update_dashboard(n):
    fetch_data()  # Fetch the latest data from MySQL

    # Aggregate the data
    agg_df = df.groupBy("hour_of_day", "user_main_genre").agg(
        F.count("*").alias("play_count")
    )
    pandas_df = agg_df.toPandas().sort_values(by='play_count', ascending=False)

    # Calculate KPIs
    top_song = df.groupBy("title").agg(F.count("*").alias("play_count")).orderBy(F.desc("play_count")).first()
    top_artist = df.groupBy("artist_name").agg(F.count("*").alias("play_count")).orderBy(F.desc("play_count")).first()
    total_play_count = df.count()
    top_genre = df.groupBy("user_main_genre").agg(F.count("*").alias("play_count")).orderBy(F.desc("play_count")).first()
    top_subscription_plan = df.groupBy("user_subscription_plan").agg(F.count("*").alias("user_count")).orderBy(F.desc("user_count")).first()
    average_song_duration = df.agg(F.avg("duration")).first()
    most_active_user = df.groupBy("user_name").agg(F.count("*").alias("play_count")).orderBy(F.desc("play_count")).first()

    # Create the figures
    fig = px.bar(pandas_df, x='hour_of_day', y='play_count', color='user_main_genre', title='Song Plays by Hour and Genre')

    age_agg_df = df.groupBy("user_age").agg(F.count("*").alias("play_count"))
    age_pandas_df = age_agg_df.toPandas()
    age_fig = px.bar(age_pandas_df, x='user_age', y='play_count', title='Play Count by Age Group')

    gender_agg_df = df.groupBy("user_gender").agg(F.count("*").alias("play_count"))
    gender_pandas_df = gender_agg_df.toPandas()
    gender_fig = px.pie(gender_pandas_df, names='user_gender', values='play_count', title='Play Count by Gender')

    subscription_agg_df = df.groupBy("user_subscription_plan").agg(F.count("*").alias("play_count"))
    subscription_pandas_df = subscription_agg_df.toPandas()
    subscription_fig = px.bar(subscription_pandas_df, x='user_subscription_plan', y='play_count', title='Play Count by Subscription Plan')

    platform_agg_df = df.groupBy("user_platform").agg(F.count("*").alias("play_count"))
    platform_pandas_df = platform_agg_df.toPandas()
    platform_fig = px.bar(platform_pandas_df, x='user_platform', y='play_count', title='Play Count by Platform')

    os_agg_df = df.groupBy("user_os").agg(F.count("*").alias("play_count"))
    os_pandas_df = os_agg_df.toPandas()
    os_fig = px.bar(os_pandas_df, x='user_os', y='play_count', title='Play Count by OS')

    return (f"{top_song['title']} (Plays: {top_song['play_count']})",
            f"{top_artist['artist_name']} (Plays: {top_artist['play_count']})",
            f"{total_play_count}",
            f"{top_genre['user_main_genre']} (Plays: {top_genre['play_count']})",
            f"{top_subscription_plan['user_subscription_plan']} (Users: {top_subscription_plan['user_count']})",
            f"{round(average_song_duration['avg(duration)'] / 60, 2)} minutes",
            f"{most_active_user['user_name']} (Plays: {most_active_user['play_count']})",
            fig, age_fig, gender_fig, subscription_fig, platform_fig, os_fig)

# 10. Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


24/08/24 14:27:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [33]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [37]:
import pandas as pd

# Assuming your data is in a DataFrame called df
numeric_df = df.select_dtypes(include=['float64', 'int64'])
correlation_matrix = numeric_df.corr()
print(correlation_matrix)


AttributeError: 'DataFrame' object has no attribute 'select_dtypes'